# Test a Tool in Isolation

The purpose of this notebook is to test a tool in isolation from the entire graph to confirm proper implementation.

**NOTE**: You must have your `GOOGLE_API_KEY` defined in your .env file. You can get this API key here: https://aistudio.google.com/app/apikey

In [3]:
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import create_react_agent
from langchain_core.runnables import RunnableLambda
from langchain.tools import tool

from currensee.core import get_model, settings

In [4]:
from dotenv import load_dotenv

load_dotenv()

True

In [15]:
from langchain_core.messages import HumanMessage
from langgraph.graph import StateGraph, START, END

from currensee.core import get_model, settings
from currensee.agents.tools.base import SupervisorState
from currensee.agents.agent_utils import summarize_all_outputs

from currensee.agents.tools.crm_tools import retrieve_client_metadata
from currensee.agents.tools.outlook_tools import produce_client_email_summary, produce_recent_client_email_summary
#, produce_recent_client_questions

from dotenv import load_dotenv
load_dotenv()

import asyncio


In [22]:
import pandas as pd
import numpy as np
from currensee.utils.db_utils import create_pg_engine
from sqlalchemy import text
from langchain_core.messages import HumanMessage
from currensee.workflows.sql_workflow.utils import create_sql_workflow
from currensee.workflows.workflow_descriptions import outlook_table_description_mapping
from currensee.core import get_model, settings
from currensee.agents.tools.base import SupervisorState
from dotenv import load_dotenv
load_dotenv()

# === Model ===
model = get_model(settings.DEFAULT_MODEL)


# === DB Connection ===
DB_NAME = 'crm_outlook'
engine = create_pg_engine(
   db_name=DB_NAME
)


def find_last_meeting_date(all_client_emails: list[str]) -> dict:
    """
        Find the last meeting date that was held with any of the 
        client emails listed above.
    """

    query_str = f"""
        SELECT  meeting_timestamp
        FROM  meeting_data
        WHERE  invitee_emails ~* '{'|'.join(all_client_emails)}' 
        ORDER BY meeting_timestamp DESC
        LIMIT 1;

    """

    last_meeting = pd.read_sql(query_str, con=engine)

    return last_meeting['meeting_timestamp'][0]


def produce_recent_client_questions(state: SupervisorState) -> dict:
    """
        Produce a the recent client questions from the most recent emails based on date recieved.
    """
    client_company = state['client_company']
    all_client_emails = state['all_client_emails']
    meeting_timestamp = state['meeting_timestamp']
    meeting_description = state['meeting_description']

    last_meeting_date = find_last_meeting_date(all_client_emails=all_client_emails)
    
    query_str = f"""
        SELECT email_body
        FROM email_data
        WHERE 
        (
            to_emails ~* '{'|'.join(all_client_emails)}' 
            OR 
            from_email ~* '{'|'.join(all_client_emails)}' 
        )
        order by email_timestamp desc
        limit 5
    """

    result = pd.read_sql(query_str, con=engine)

    recent_emails = list(result['email_body'])

    recent_email_str = '\n'.join(recent_emails)

    ###### SUMMARIZE HERE #########
    summary_prompt = f""" 
    PROMPT

    You are reviewing a series of past emails exchanged between the client {client_company} and Bankwell Financial to identify client questions.
    
    Extract any questions asked by {client_company} or their representatives. 
        - Do not include questions about availability, meeting times, or scheduling logistics.
        - Only include questions asked by {client_company} or its representative, not by the Bankwell Financial Employee.
        - Use verbatim quotes from the emails where possible. You may lightly paraphrase for clarity.
        - Present this as a numbered list. 
        
    Format your response like this:
    
    Client Questions:
    1. "[Exact client question]"
    2. "[Exact client question]"
    3. "[Paraphrased question if needed]"
    ...
    
    Email Thread to Analyze:
    {recent_email_str}
    
    """

    # Create the messages to pass to the model
    messages = [
        HumanMessage(content=summary_prompt)
    ]
    
    # Use the 'invoke' method for summarization
    recent_client_questions = model.invoke(messages)


    ############# Return the new state ###############

    new_state = state.copy()
    new_state['last_meeting_timestamp'] = last_meeting_date
    new_state['recent_client_questions'] = recent_client_questions.content

    
    return new_state


In [23]:
# Define the gemini model for the tool
model = get_model(settings.DEFAULT_MODEL)

complete_graph_test = StateGraph(SupervisorState)

complete_graph_test.add_node("retrieve_client_metadata", retrieve_client_metadata)
complete_graph_test.add_node("produce_recent_outlook_summary", produce_recent_client_email_summary)
complete_graph_test.add_node("produce_recent_client_questions", produce_recent_client_questions)

complete_graph_test.add_edge(START, "retrieve_client_metadata")
complete_graph_test.add_edge("retrieve_client_metadata", "produce_recent_outlook_summary")
complete_graph_test.add_edge("produce_recent_outlook_summary", "produce_recent_client_questions")
complete_graph_test.add_edge("produce_recent_client_questions", END)

compiled_graph_test = complete_graph_test.compile()

In [24]:
init_state = {
        'client_name': 'Jessica Palmer',
        'client_email': 'jessica.palmer@hasbro.com',
        'meeting_timestamp': '2024-02-04 14:30:00',
        'meeting_description': 'Hasbro - Discuss Portfolio Diversification and Bond Strategy',
        'report_length': 'short'   # 'short' , 'medium' , 'long' (default)
    }
result = compiled_graph_test.invoke(init_state)
summary = result['recent_client_questions']
full_report = summary 
print(full_report)

Client Questions:

1. "Do you believe that current market trends will continue regarding small-cap consumer entertainment?"
2. "Also, I am curious about our allocation in bond funds. Given current inflation, what is the likely impact on the bond market, i.e. should we consider moving further out of bonds?"
